# Building own HMM PoS Tagger

The Hidden Markov Model (HMM) is an extension of the Markov process used to model phenomena where the states are hidden or latent, but they emit observations. For instance, in a speech recognition system like a speech-to-text converter, the states represent the actual text words to predict, but they are not directly observable (i.e., the states are hidden). Rather, you only observe the speech (audio) signals corresponding to each word and need to deduce the states using the observations.

Similarly, in POS tagging, you observe the words in a sentence, but the POS tags themselves are hidden. Thus, the POS tagging task can be modeled as a Hidden Markov Model with the hidden states representing POS tags that emit observations, i.e., words.

The hidden states emit observations with a certain probability. Therefore, Hidden Markov Model has emission probabilities, which represent the probability that a particular state emits a given observation. Along with the transition and initial state probabilities, these emission probabilities are used to model HMMs.

The figure below illustrates the emission and transition probabilities for a hidden Markov process with three hidden states and four observations.


v![HMM](https://wisdomml.in/wp-content/uploads/2023/04/HMM.png)



**Documentation**
- Coding:
[Complete Example](https://wisdomml.in/hidden-markov-model-hmm-in-nlp-python/#Viterbi_Algorithm-2)
- [A deep dive into part-of-speech tagging using the Viterbi algorithm](https://www.freecodecamp.org/news/a-deep-dive-into-part-of-speech-tagging-using-viterbi-algorithm-17c8de32e8bc/)

## Treebank Tagged Corpus



In [ ]:
#Importing libraries
import nltk
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize


In [ ]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents())
# first few tagged sentences
print(wsj[:40])

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.')], [('A', 'DT'), ('f

### Train/Test Dataset

we will split the dataset into a 70:30 ratio i.e., 70% of the data for the training set and the rest 30% for the test set.

In [ ]:
train_set, test_set = train_test_split(wsj,test_size=0.3)
print(len(train_set))
print(len(test_set))
print(train_set[:40])

2739
1175
[[('Previously', 'RB'), (',', ','), ('watch', 'NN'), ('imports', 'NNS'), ('were', 'VBD'), ('denied', 'VBN'), ('*-37', '-NONE-'), ('such', 'JJ'), ('duty-free', 'JJ'), ('treatment', 'NN'), ('.', '.')], [('While', 'IN'), ('many', 'JJ'), ('problems', 'NNS'), ('would', 'MD'), ('attend', 'VB'), ('a', 'DT'), ('restructuring', 'NN'), ('of', 'IN'), ('Columbia', 'NNP'), (',', ','), ('investors', 'NNS'), ('say', 'VBP'), ('0', '-NONE-'), ('Mr.', 'NNP'), ('Spiegel', 'NNP'), ('is', 'VBZ'), ('mulling', 'VBG'), ('such', 'PDT'), ('a', 'DT'), ('plan', 'NN'), ('0', '-NONE-'), ('*', '-NONE-'), ('to', 'TO'), ('mitigate', 'VB'), ('Columbia', 'NNP'), ("'s", 'POS'), ('junk', 'NN'), ('problems', 'NNS'), ('*T*-1', '-NONE-'), ('.', '.')], [('He', 'PRP'), ('described', 'VBD'), ('the', 'DT'), ('situation', 'NN'), ('as', 'IN'), ('``', '``'), ('an', 'DT'), ('escrow', 'NN'), ('problem', 'NN'), (',', ','), ('a', 'DT'), ('timing', 'NN'), ('issue', 'NN'), (',', ','), ("''", "''"), ('which', 'WDT'), ('he', 'PRP

In [ ]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)
train_tagged_words[:5]

[('Previously', 'RB'),
 (',', ','),
 ('watch', 'NN'),
 ('imports', 'NNS'),
 ('were', 'VBD')]

### Create token and tag set

Next, we will create a tokens variable that will contain all the tokens from the train_tagged_words. Furthermore, we also need to create a vocabulary and set of all unique tags in the training data.

In [ ]:
# tokens
tokens = [pair[0] for pair in train_tagged_words]
# vocabulary
V = set(tokens)
print("Total vocabularies: ",len(V))
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print("Total tags: ",len(T))


Total vocabularies:  10270
Total tags:  46


## Coding HMM


### Emission probabilities

In [ ]:
import numpy as np

In [ ]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [ ]:
w_given_t.shape

(46, 10270)

In [ ]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)

    return (count_w_given_tag, count_tag)

In [ ]:
# examples
# large
print("\n", "large")
print(word_given_tag('large', 'JJ'))
print(word_given_tag('large', 'VB'))
print(word_given_tag('large', 'NN'), "\n")
# will
print("\n", "will")
print(word_given_tag('will', 'MD'))
print(word_given_tag('will', 'NN'))
print(word_given_tag('will', 'VB'))
# book
print("\n", "book")
print(word_given_tag('book', 'NN'))
print(word_given_tag('book', 'VB'))


 large
(22, 4101)
(0, 1793)
(0, 9266) 


 will
(201, 662)
(0, 9266)
(0, 1793)

 book
(4, 9266)
(1, 1793)


### Transition Probabilities

In [ ]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# examples
print(t2_given_t1(t2='NNP', t1='JJ'))
print(t2_given_t1('NN', 'JJ'))
print(t2_given_t1('NN', 'DT'))
print(t2_given_t1('NNP', 'VB'))
print(t2_given_t1(',', 'NNP'))
print(t2_given_t1('PRP', 'PRP'))
print(t2_given_t1('VBG', 'NNP'))

(138, 4101)
(1860, 4101)
(2715, 5733)
(61, 1793)
(996, 6541)
(3, 1178)
(4, 6541)


In [ ]:
#Please note P(tag|start) is same as P(tag|'.')
print(t2_given_t1('DT', '.'))
print(t2_given_t1('VBG', '.'))
print(t2_given_t1('NN', '.'))
print(t2_given_t1('NNP', '.'))

(570, 2707)
(12, 2707)
(108, 2707)
(501, 2707)


Next, we will create a transition matrix of tags of dimension txt

In [ ]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

tags_matrix

array([[0.03050398, 0.04376658, 0.        , ..., 0.00132626, 0.        ,
        0.        ],
       [0.0034138 , 0.06120458, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01277955, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02061856, 0.35051546, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07692308, 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [ ]:
import pandas as pd
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,VBN,JJ,WDT,-LRB-,$,VBG,VBP,IN,NN,.,...,CD,-RRB-,WP,MD,'',:,PRP$,RBR,LS,WP$
VBN,0.030504,0.043767,0.000000,0.000000,0.004642,0.010610,0.000000,0.081565,0.066976,0.009947,...,0.007958,0.000000,0.002653,0.000000,0.003316,0.002653,0.013926,0.001326,0.000000,0.000000
JJ,0.003414,0.061205,0.000000,0.000732,0.002195,0.005121,0.000488,0.057303,0.453548,0.019507,...,0.020727,0.000000,0.000244,0.000244,0.003414,0.003414,0.000000,0.000000,0.000000,0.000000
WDT,0.000000,0.012780,0.000000,0.000000,0.000000,0.000000,0.000000,0.003195,0.003195,0.000000,...,0.003195,0.000000,0.000000,0.003195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
-LRB-,0.043956,0.021978,0.000000,0.000000,0.175824,0.010989,0.000000,0.087912,0.054945,0.000000,...,0.065934,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
$,0.000000,0.014028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.985972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VBG,0.019305,0.068533,0.000000,0.000000,0.008687,0.002896,0.000000,0.122587,0.150579,0.017375,...,0.023166,0.000965,0.001931,0.000000,0.000965,0.001931,0.032819,0.002896,0.000000,0.000000
VBP,0.172571,0.073143,0.000000,0.002286,0.003429,0.070857,0.000000,0.101714,0.025143,0.008000,...,0.009143,0.000000,0.002286,0.000000,0.001143,0.001143,0.008000,0.004571,0.000000,0.000000
IN,0.003648,0.097607,0.003648,0.000000,0.027575,0.002918,0.000146,0.016633,0.109425,0.003210,...,0.062008,0.000000,0.001751,0.000000,0.000146,0.000146,0.034724,0.000730,0.000000,0.000146
NN,0.007123,0.006907,0.008310,0.001403,0.000108,0.007231,0.003561,0.239046,0.127995,0.101554,...,0.004749,0.001511,0.003238,0.013922,0.005180,0.015433,0.000216,0.000755,0.000000,0.000108
.,0.001847,0.036572,0.000739,0.004064,0.001108,0.004433,0.000000,0.123384,0.039897,0.000000,...,0.007758,0.004433,0.003325,0.000369,0.059106,0.002216,0.005541,0.000739,0.001478,0.000000


**Viterbi algorithm**

The purpose of the Viterbi algorithm is to make an inference based on a trained model and some observed data. It works by asking a question: given the trained parameter matrices and data, what is the choice of states such that the joint probability reaches maximum? In other words, what is the most likely choice given the data and the trained model? This statement can be visualized as the following formula, and obviously, the answer depends on the data!

Ref:
- [Viterbi](https://medium.com/analytics-vidhya/viterbi-algorithm-for-prediction-with-hmm-part-3-of-the-hmm-series-6466ce2f5dc6)
- Further reading: [slide](https://www.cl.cam.ac.uk/teaching/1718/MLRD/slides/slides9.pdf)


In [ ]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

now, we will tag the test sentences using the Viterbi algorithm

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
## Testing
sentence_test = 'This is the test sentence to be tagged using HMM'
words = nltk.word_tokenize(sentence_test)
tagged_seq = Viterbi(words)
print(tagged_seq)

[('This', 'DT'), ('is', 'VBZ'), ('the', 'DT'), ('test', 'NN'), ('sentence', 'NN'), ('to', 'TO'), ('be', 'VB'), ('tagged', 'VBN'), ('using', 'VBG'), ('HMM', 'VBN')]
